# Практические занятия №12-13 (часть 2)

### Фаззинг (Тестирование на случайных структурированных входных данных)

**4.1.** (1 балл)

Продолжение раздела "Контрактное программирование" из части 1. Напишите тест, доказывающий, что переданная на вход функция коммутативна. Используйте модуль `hypothesis`. Запустите тесты для реализованных ранее функций `euclidean_distance` и `manhattan_distance`.  

### Инварианты классов

**4.2.** (1 балл)

В приведённом ниже коде для работы с банковским счётом есть следующие проблемы:

1. Снятие суммы без проверки баланса.
2. Снятие/добавление отрицательной суммы на счет.
3. В поле `balance` может быть записано всё что угодно.

Решите их путём добавления инвариантов класса `deal.inv`. Напишите тесты для всех методов класса `BankAccount`. Обязательно напишите тест, проверяющий, что баланс равен 0.2 после зачисления 0.3 и снятия 0.1 средств.

In [ ]:
class BankAccount:
    def __init__(self, balance=0):
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        return f"{amount} средств успешно зачислены на счёт."

    def withdraw(self, amount):
        self.balance -= amount
        return f"{amount} средств успешно сняты с счёта."

    def check_balance(self):
        return f"Баланс счёта: {self.balance}"

In [2]:
from deal import inv

class BankAccount:
    @inv(lambda self: self.balance >= 0)
    def __init__(self, balance=0):
        self.balance = balance

    @inv(lambda self, amount: amount >= 0)
    def deposit(self, amount):
        self.balance += amount
        return f"{amount} средств успешно зачислены на счёт."

    @inv(lambda self, amount: amount >= 0 and amount <= self.balance)
    def withdraw(self, amount):
        self.balance -= amount
        return f"{amount} средств успешно сняты с счёта."

    @inv(lambda self: isinstance(self.balance, (int, float)))
    def check_balance(self):
        return f"Баланс счёта: {self.balance}"


# Тесты
def test_bank_account():
    account = BankAccount()
    assert account.check_balance() == "Баланс счёта: 0"
    
    account.deposit(0.3)
    assert account.check_balance() == "Баланс счёта: 0.3"
    
    account.withdraw(0.1)
    assert account.check_balance() == "Баланс счёта: 0.2"
    
    assert account.deposit(-0.1) == AssertionError
    assert account.withdraw(-0.1) == AssertionError
    assert account.withdraw(0.3) == AssertionError

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

### Мутационное тестирование

**4.3.** (1 балл)

Суть мутационного тестирования (mutation testing) состоит в привнесении ошибок ("мутаций") в исходную программу (тем самым создаются "программы-мутанты"), с тем, чтобы оценить, сможет ли существующий набор тестов эти ошибки определить. Если набор тестов успешно выполняется на программе-мутанте, то это означает, что используемые тесты нуждаются в развитии.

Продолжение раздела "Библиотеки pytest и coverage (задача 2.1.)". Намеренно внесите минимум по одному ошибочному изменению в коде методов следующих классов:


1.   User - `authenticate()`, `authorize()`.
2.   Project - `add_task()`, `remove_task()`, `find_task_by_name()`, `find_task_by_start_date()`, `is_completed()`.
3.   Task - `change_status()`, `remove_performer()`.

Проверьте существующий набор тестов для задачи 2.1. на обновлённом коде. Проходят ли какие-то тесты? Попытайтесь добиться обратного эффекта: ни один тест, проверяющий указанные выше методы, не должен проходить. Проверку возможно осуществлять также через `pytest` и `coverage`.